# GO tool for Image Retrieval and Metadata Organization

In this notebook we have explored existing open-source tools that allow one to search and retrieve satellite imagery and to store the corresponding metadata for subsequent queries.  

## 1. Image Search

Python-library or command line tool `sat-search`.

In [1]:
from satsearch import Search

In [2]:
search = Search.search(
    url="https://earth-search.aws.element84.com/v0",
    collections=["sentinel-s2-l2a-cogs"],
    datetime="2018-02-25/2018-04-25",
    # query sentinel-2 tile 5VNK
    query=[
        "sentinel:utm_zone=5",
        "sentinel:latitude_band=V",
        "sentinel:grid_square=NK"
    ]
)
items = search.items()
len(items)

29

In [3]:
summary = items.summary(['date', 'id', 'eo:cloud_cover'])
print(summary)

Items (29):
date                      id                        eo:cloud_cover            
2018-04-24                S2B_5VNK_20180424_0_L2A   76.86                     
2018-04-23                S2A_5VNK_20180423_0_L2A   84.82                     
2018-04-21                S2B_5VNK_20180421_0_L2A   83.26                     
2018-04-19                S2A_5VNK_20180419_0_L2A   4.97                      
2018-04-18                S2B_5VNK_20180418_0_L2A   87.24                     
2018-04-16                S2A_5VNK_20180416_0_L2A   38.82                     
2018-04-14                S2B_5VNK_20180414_0_L2A   36.32                     
2018-04-13                S2A_5VNK_20180413_0_L2A   19.75                     
2018-04-11                S2B_5VNK_20180411_0_L2A   87.31                     
2018-04-09                S2A_5VNK_20180409_0_L2A   42.17                     
2018-04-08                S2B_5VNK_20180408_0_L2A   33.05                     
2018-04-06                S2A_5VNK_20180

In [4]:
# save outcome of the search to GeoJSON format
items.save('results.json')

## 2. Download
For images in public repositories (Landsat 8, Sentinel 2 L2A) direct download is possible via `sat-stac`. For Sentinel 2 L1C, Ou's tool from Google Cloud Storage.

In [5]:
from satstac import ItemCollection

filename_template = '${collection}/${sentinel:utm_zone}/${sentinel:latitude_band}/${sentinel:grid_square}/${id}'

# assets to retrieve
assets = ('metadata', 'B01')

In [6]:
# load search results
items = ItemCollection.open('results.json')

for item in items:
    for asset in assets:
        item.download(asset, filename_template=filename_template)

## 3. Store Metadata

- Parse `GeoJSON` and export it to SQL datatabase e.g. via `geopandas` (NOTE: only subset of attributes are parsed, e.g. assets are excluded):

In [7]:
import geopandas as gpd

In [8]:
gdf = gpd.read_file('results.json')
gdf.head()

,id,datetime,platform,constellation,gsd,data_coverage,view:off_nadir,eo:cloud_cover,proj:epsg,sentinel:latitude_band,sentinel:grid_square,sentinel:sequence,sentinel:product_id,created,updated,sentinel:valid_cloud_cover,sentinel:utm_zone,sentinel:data_coverage,geometry
0,S2B_5VNK_20180424_0_L2A,2018-04-24T21:55:27+00:00,sentinel-2b,sentinel-2,10,43.06,0,76.86,32605,V,NK,0,S2B_MSIL2A_20180424T215529_N0001_R029_T05VNK_2...,2020-09-27T01:37:37.775002+00:00,2020-09-27T01:37:37.775002+00:00,True,5,43.06,"POLYGON ((-152.53008 62.14319, -153.00036 62.1..."
1,S2A_5VNK_20180423_0_L2A,2018-04-23T21:35:31+00:00,sentinel-2a,sentinel-2,10,99.33,0,84.82,32605,V,NK,0,S2A_MSIL2A_20180423T213531_N0001_R086_T05VNK_2...,2020-10-02T06:23:26.086000+00:00,2020-10-02T06:23:26.086000+00:00,True,5,99.33,"POLYGON ((-150.89497 62.12799, -153.00036 62.1..."
2,S2B_5VNK_20180421_0_L2A,2018-04-21T21:45:28+00:00,sentinel-2b,sentinel-2,10,100.00,0,83.26,32605,V,NK,0,S2B_MSIL2A_20180421T214529_N0001_R129_T05VNK_2...,2020-09-08T11:58:18.973000+00:00,2020-09-08T11:58:18.973000+00:00,True,5,100.00,"POLYGON ((-150.89497 62.12799, -153.00036 62.1..."
3,S2A_5VNK_20180419_0_L2A,2018-04-19T21:55:30+00:00,sentinel-2a,sentinel-2,10,42.64,0,4.97,32605,V,NK,0,S2A_MSIL2A_20180419T215531_N0001_R029_T05VNK_2...,2020-09-26T22:03:57.210999+00:00,2020-09-26T22:03:57.210999+00:00,True,5,42.64,"POLYGON ((-152.53548 62.14321, -153.00036 62.1..."
4,S2B_5VNK_20180418_0_L2A,2018-04-18T21:35:27+00:00,sentinel-2b,sentinel-2,10,99.32,0,87.24,32605,V,NK,0,S2B_MSIL2A_20180418T213529_N0001_R086_T05VNK_2...,2020-09-26T18:44:51.902000+00:00,2020-09-26T18:44:51.902000+00:00,True,5,99.32,"POLYGON ((-150.89497 62.12799, -153.00036 62.1..."


In [9]:
# export table to database
# gdf.to_sql('data', engine)

# load it back as data-frame
# gpd.read_postgis('data', engine)

- Organize metadata in local STAC catalog, using `sat-stac` (integrated with `sat-search`) or [`pystac`](https://pystac.readthedocs.io/en/latest/):

In [10]:
import pathlib

from satstac import Catalog, Collection, ItemCollection

# set paths
path_template = '${sentinel:utm_zone}/${sentinel:latitude_band}/${sentinel:grid_square}/${id}'
root_path = pathlib.Path('.').absolute()

In [11]:
# load search results
items = ItemCollection.open('results.json')

# create local catalog
cat = Catalog({'id': 'Eratosthenes', 'description': 'Catalog for Eratosthenes GO project'})
cat = cat.save('Eratosthenes/catalog.json')

# loop over items found
for item in items:
    col = item.collection()
    
    # if collection not present, add it to catalog
    if col.id not in [c.id for c in cat.catalogs()]:
        cat.add_catalog(col)
        
    # the asset download can be integrated here as well!
    for asset in assets:
        asset_template = (root_path/'${collection}'/path_template).as_posix()
        path = item.download(asset, filename_template=asset_template)
        # modify asset href to local copy 
        item.asset(asset)['href'] = path
        
    # finally add item to catalog
    item_template = '.'.join([path_template, 'json'])
    col.add_item(item, item_template)  

## 4. Query

Export (subsection of) catalog to `GeoJSON`, import it in `geopandas`.

In [12]:
from geopandas import GeoDataFrame

In [13]:
# export all items from the catalog
cat = Catalog.open('Eratosthenes/catalog.json')
items = ItemCollection([i for i in cat.items()])

# import in GeoDataFrame
gdf = GeoDataFrame.from_features(items.geojson())
gdf['id'] = [item.id for item in items]
gdf.head()

,geometry,datetime,platform,constellation,instruments,gsd,data_coverage,view:off_nadir,eo:cloud_cover,proj:epsg,sentinel:latitude_band,sentinel:grid_square,sentinel:sequence,sentinel:product_id,created,updated,sentinel:valid_cloud_cover,sentinel:utm_zone,sentinel:data_coverage,id
0,"POLYGON ((-152.53008 62.14319, -153.00036 62.1...",2018-04-24T21:55:27Z,sentinel-2b,sentinel-2,[msi],10,43.06,0,76.86,32605,V,NK,0,S2B_MSIL2A_20180424T215529_N0001_R029_T05VNK_2...,2020-09-27T01:37:37.775Z,2020-09-27T01:37:37.775Z,True,5,43.06,S2B_5VNK_20180424_0_L2A
1,"POLYGON ((-150.89497 62.12799, -153.00036 62.1...",2018-04-23T21:35:31Z,sentinel-2a,sentinel-2,[msi],10,99.33,0,84.82,32605,V,NK,0,S2A_MSIL2A_20180423T213531_N0001_R086_T05VNK_2...,2020-10-02T06:23:26.086Z,2020-10-02T06:23:26.086Z,True,5,99.33,S2A_5VNK_20180423_0_L2A
2,"POLYGON ((-150.89497 62.12799, -153.00036 62.1...",2018-04-21T21:45:28Z,sentinel-2b,sentinel-2,[msi],10,100.00,0,83.26,32605,V,NK,0,S2B_MSIL2A_20180421T214529_N0001_R129_T05VNK_2...,2020-09-08T11:58:18.973Z,2020-09-08T11:58:18.973Z,True,5,100.00,S2B_5VNK_20180421_0_L2A
3,"POLYGON ((-152.53548 62.14321, -153.00036 62.1...",2018-04-19T21:55:30Z,sentinel-2a,sentinel-2,[msi],10,42.64,0,4.97,32605,V,NK,0,S2A_MSIL2A_20180419T215531_N0001_R029_T05VNK_2...,2020-09-26T22:03:57.211Z,2020-09-26T22:03:57.211Z,True,5,42.64,S2A_5VNK_20180419_0_L2A
4,"POLYGON ((-150.89497 62.12799, -153.00036 62.1...",2018-04-18T21:35:27Z,sentinel-2b,sentinel-2,[msi],10,99.32,0,87.24,32605,V,NK,0,S2B_MSIL2A_20180418T213529_N0001_R086_T05VNK_2...,2020-09-26T18:44:51.902Z,2020-09-26T18:44:51.902Z,True,5,99.32,S2B_5VNK_20180418_0_L2A


In [14]:
mask = (gdf['eo:cloud_cover'] < 10) & (gdf.platform == 'sentinel-2b')
gdf_subset = gdf[mask]
gdf_subset

,geometry,datetime,platform,constellation,instruments,gsd,data_coverage,view:off_nadir,eo:cloud_cover,proj:epsg,sentinel:latitude_band,sentinel:grid_square,sentinel:sequence,sentinel:product_id,created,updated,sentinel:valid_cloud_cover,sentinel:utm_zone,sentinel:data_coverage,id
17,"POLYGON ((-152.51967 62.14316, -153.00036 62.1...",2018-03-25T21:55:23Z,sentinel-2b,sentinel-2,[msi],10,NaN,0,0.0,32605,V,NK,1,S2B_MSIL2A_20180325T215529_N0001_R029_T05VNK_2...,2020-09-28T19:40:26.599Z,2020-09-28T19:40:26.599Z,False,5,43.51,S2B_5VNK_20180325_1_L2A
25,"POLYGON ((-152.53328 62.14320, -153.00036 62.1...",2018-03-05T21:55:19Z,sentinel-2b,sentinel-2,[msi],10,42.77,0,0.0,32605,V,NK,0,S2B_MSIL2A_20180305T215519_N0001_R029_T05VNK_2...,2020-09-08T08:56:12.672Z,2020-09-08T08:56:12.672Z,True,5,42.77,S2B_5VNK_20180305_0_L2A


In [15]:
ids = [id for id in gdf_subset.id]
ids

['S2B_5VNK_20180325_1_L2A', 'S2B_5VNK_20180305_0_L2A']

## 5. Import

Apart from the 'manual' search over the local filesystem, `intake` (with plugins) allows to load assets from catalog. NOTE: the `intake-stac` plugin is still under development.

In [16]:
import intake

In [17]:
col = intake.open_stac_collection('Eratosthenes/sentinel-s2-l2a-cogs/catalog.json')

In [18]:
col[ids[0]].B02.metadata

{'title': 'Band 2 (blue)',
 'type': 'image/tiff; application=geotiff; profile=cloud-optimized',
 'roles': ['data'],
 'gsd': 10,
 'eo:bands': [{'name': 'B02',
   'common_name': 'blue',
   'center_wavelength': 0.4966,
   'full_width_half_max': 0.098}],
 'href': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/5/V/NK/2018/3/S2B_5VNK_20180325_1_L2A/B02.tif',
 'proj:shape': [10980, 10980],
 'proj:transform': [10, 0, 499980, 0, -10, 7000020, 0, 0, 1],
 'plots': {'geotiff': {'kind': 'image',
   'x': 'x',
   'y': 'y',
   'frame_width': 500,
   'data_aspect': 1,
   'rasterize': True,
   'dynamic': True,
   'cmap': 'viridis'}},
 'catalog_dir': ''}

In [19]:
ds = col[ids[0]].B01.to_dask()
ds

<xarray.DataArray (band: 1, y: 1830, x: 1830)>
dask.array<open_rasterio-8050c971f1d2f814ad8277c36f6e6060<this-array>, shape=(1, 1830, 1830), dtype=uint16, chunksize=(1, 1830, 1830), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 7e+06 7e+06 7e+06 7e+06 ... 6.89e+06 6.89e+06 6.89e+06
  * x        (x) float64 5e+05 5.001e+05 5.001e+05 ... 6.097e+05 6.098e+05
Attributes:
    transform:           (60.0, 0.0, 499980.0, 0.0, -60.0, 7000020.0)
    crs:                 +init=epsg:32605
    res:                 (60.0, 60.0)
    is_tiled:            1
    nodatavals:          (0.0,)
    scales:              (1.0,)
    offsets:             (0.0,)
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  AVERAGE

In [20]:
chunks = dict(band=1, x=512, y=512)
ds2 = col[ids[1]].B01(chunks=chunks).to_dask()
ds2

<xarray.DataArray (band: 1, y: 1830, x: 1830)>
dask.array<open_rasterio-8d5d267cb08fd640cb862944dc88f066<this-array>, shape=(1, 1830, 1830), dtype=uint16, chunksize=(1, 512, 512), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 7e+06 7e+06 7e+06 7e+06 ... 6.89e+06 6.89e+06 6.89e+06
  * x        (x) float64 5e+05 5.001e+05 5.001e+05 ... 6.097e+05 6.098e+05
Attributes:
    transform:           (60.0, 0.0, 499980.0, 0.0, -60.0, 7000020.0)
    crs:                 +init=epsg:32605
    res:                 (60.0, 60.0)
    is_tiled:            1
    nodatavals:          (0.0,)
    scales:              (1.0,)
    offsets:             (0.0,)
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  AVERAGE